In [22]:
from modelHandler import ModelHandler
import pandas as pd  
import numpy as np 
import sklearn as sk

In [82]:
df = pd.read_csv('../data/Correlation01.csv', index_col = 0)

## Group by Position

Partiendo de la hipotesis que goles y asistencias tienen más peso para delanteros y mediocentros que para defensas y obviamente que para porteros, la división de estos grupos es fundamental.

Creemos que es interesante realizar un modelo para cada posición del campo de futbol. Elegimos realizar una primera aproximación con modelos para:
- Portero
- Defensa i mediocentro defensivo
- Mediocentros y bandas
- Delanteros y extremos

Otra configuración podria ser juntar mediocentros y delanteros. 

In [83]:
### group positon
tre = { 'Goalkeeper' : 0,
    'DefensiveMidfield': 1,'Defender' : 1,
 'LeftMidfield':2 , 'CentralMidfield':2, 'Midfielder':2, 'AttackingMidfield':2,'RightMidfield' : 2,
   'SecondStriker':3, 'Forward':3,'LeftWinger':3,'RightWinger' : 3}

df['position'] = df['position'].apply(lambda x: tre[x])

In [84]:
df.describe()

,price,yellow_card_champ,yellow_card_cup,goal_continent,assist_continent,yellow_card_continent,score_goal_cup,score_goal_champ,score_goals_selection,score_assist_cup,score_assist_champ,score_selections_nation,league,age,position
count,8.854000e+03,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000,8854.000000
mean,1.612051e+06,14.754009,0.857353,0.479105,0.459002,0.602778,0.863564,11.556472,0.580416,0.703185,9.018071,5.707364,37.212333,26.045968,1.684098
std,5.881306e+06,17.824661,1.992781,2.190870,1.823347,1.887254,2.187195,18.585831,2.522602,1.857391,14.890667,14.846162,18.112519,4.564612,1.040544
min,2.500000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,0.000000
25%,1.000000e+05,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,23.000000,1.000000
50%,3.000000e+05,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,3.000000,0.000000,38.000000,26.000000,2.000000
75%,8.000000e+05,21.000000,1.000000,0.000000,0.000000,0.000000,1.000000,15.000000,0.000000,0.000000,12.000000,4.000000,50.000000,29.000000,2.000000
max,1.000000e+08,142.000000,26.000000,121.000000,44.000000,28.000000,44.000000,423.000000,85.000000,23.000000,174.000000,176.000000,70.000000,46.000000,3.000000


## values

## Model handler

# For each position

In [12]:
def model_for_position(df):
    pos_mod = {}
    pos = list(df.position.unique())
    target = 'price'
    for p in pos:
        tmp = df[df.position == p]
        pos_mod[p] = ModelHandler(X = tmp.drop(target, axis = 1).to_numpy(), Y = tmp[target].to_numpy(), model = 'XGB',scale = True)
    return pos_mod

In [13]:
models = model_for_position(df)

In [14]:
models

{0: <modelHandler.ModelHandler at 0x7f6d4016a790>,
 3: <modelHandler.ModelHandler at 0x7f6d4016aa30>,
 2: <modelHandler.ModelHandler at 0x7f6d4016a160>,
 1: <modelHandler.ModelHandler at 0x7f6d4016ac40>}

In [16]:
for model in models.values():
    model.fit(with_score = False)

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[INFO] The best parameters are {'colsample_bytree': 0.5, 'learning_rate': 0.001, 'max_depth': 24, 'n_estimators': 1000, 'subsample': 1.0}
[INFO] The best score is -844596.7693
Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=6, n_estimators=1000, subsample=0.5; total time=   5.3s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=6, n_estimators=1000, subsample=1.0; total time=   4.9s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=12, n_estimators=1000, subsample=0.5; total time=   6.8s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=12, n_estimators=1000, subsample=1.0; total time=   6.9s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=24, n_estimators=1000, subsample=0.5; total time=   7.1s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=24, n_estimators=1000, subsample=1.0; total time=   8.7s


/home/ubuntu/anaconda3/envs/rubix/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=12, n_estimators=1000, subsample=1.0; total time=  10.8s
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=24, n_estimators=1000, subsample=0.5; total time=  14.5s
[CV] END colsample_bytree=1.0, learning_rate=0.001, max_depth=24, n_estimators=1000, subsample=1.0; total time=  17.4s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=6, n_estimators=1000, subsample=0.5; total time=   6.6s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=6, n_estimators=1000, subsample=1.0; total time=   6.4s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=12, n_estimators=1000, subsample=0.5; total time=  10.7s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=12, n_estimators=1000, subsample=1.0; total time=   9.7s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=24, n_estimators=1000, subsample=0.5; total time=  16.5s
[CV] END colsample_bytree=0.5, learning_rate=0.3, max_depth=

In [30]:
models[0].grid.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.001,
 'max_depth': 24,
 'n_estimators': 1000,
 'subsample': 1.0}

In [109]:
from xgboost import XGBRegressor
def model_for_position(df):
    pos_mod = {}
    pos_data_train = {}
    pos_data_test = {}
    pos = list(df.position.unique())
    target = 'price'
    for p in pos:
        tmp = df[df.position == p]
        models[p].grid.best_params_['n_estimators'] = 1000
        models[p].grid.best_params_['max_depth'] = 64

        pos_mod[p] = XGBRegressor(**models[p].grid.best_params_)
        X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(tmp.drop(target, axis = 1).to_numpy(), tmp[target].to_numpy(), test_size=0.20, random_state=42)
        pos_data_train[p] = (X_train, y_train)
        pos_data_test[p] = (X_test, y_test)

    return pos_mod, pos_data_train, pos_data_test

In [110]:
pos_mod, pos_data_train, pos_data_test = model_for_position(df)

In [111]:
from sklearn.metrics import mean_squared_error


In [112]:
def train():
    for k in pos_mod:
        pos_mod[k].fit(pos_data_train[k][0], pos_data_train[k][1])
        print(f"INFO class {k} mse on train: {mean_squared_error(pos_data_train[k][1], pos_mod[k].predict(pos_data_train[k][0]))}")
        print(f"INFO class {k} mse on test: {mean_squared_error(pos_data_test[k][1], pos_mod[k].predict(pos_data_test[k][0]))}")

In [ ]:
train()

INFO class 0 mse on train: 9965955538301.834
INFO class 0 mse on test: 6754330757890.502
INFO class 3 mse on train: 17144979790920.572
INFO class 3 mse on test: 45156881764925.39


In [ ]:
for t,p in zip(pos_data_test[0][1], pos_mod[0].predict(pos_data_test[0][0])):
    print(t,p, t-p)